In [1]:
import pandas as pd
import numpy as np
import json
import geopy.distance
import matplotlib.pyplot as plt
%matplotlib inline

# Using Divvy data

## Load and concat the Divvy datasets

In [ ]:
# divvy1 = pd.read_csv('data/Divvy_Trips_2017_Q1Q2/Divvy_Trips_2017_Q1.csv', parse_dates=['start_time', 'end_time'])
# divvy2 = pd.read_csv('data/Divvy_Trips_2017_Q1Q2/Divvy_Trips_2017_Q2.csv', parse_dates=['start_time', 'end_time'])
# divvy3 = pd.read_csv('data/Divvy_Trips_2017_Q3Q4/Divvy_Trips_2017_Q3.csv', parse_dates=['start_time', 'end_time'])
# divvy4 = pd.read_csv('data/Divvy_Trips_2017_Q3Q4/Divvy_Trips_2017_Q4.csv', parse_dates=['start_time', 'end_time'])

# divvy = pd.concat([divvy1, divvy2, divvy3, divvy4], ignore_index=True)

# divvy.to_csv('data/divvy_2017.csv')

In [ ]:
divvy = pd.read_csv('data/divvy_2017.csv')

In [ ]:
divvy

In [ ]:
station_list = set(list(divvy['from_station_name'].unique()) + list(divvy['to_station_name'].unique()))
station_list

In [ ]:
len(station_list)

## Load Divvy station info for GPS coordinates

In [ ]:
with open('data/stations.json') as json_data:
    station_data = json.load(json_data)

In [ ]:
station_data['stationBeanList']

In [ ]:
stations = [station['stationName'] for station in station_data['stationBeanList']]
latitude = [station['latitude'] for station in station_data['stationBeanList']]
longitude = [station['longitude'] for station in station_data['stationBeanList']]

In [ ]:
len(stations)

In [ ]:
unknown = []
for station in station_list:
    if station not in stations:
        unknown.append(station)
unknown

In [ ]:
station_gps = pd.DataFrame({'station_name': stations, 'latitude': latitude, 'longitude': longitude})

In [ ]:
station_gps.head()

In [ ]:
def gps_lookup(location):
    match = (station_gps['station_name'] == location)
    coord = station_gps['latitude'][match]
    if len(coord) > 0:
        return pd.Series([coord.values[0], station_gps['longitude'][match].values[0]])
    else:
        return pd.Series([np.nan, np.nan])

In [ ]:
divvy[['from_station_latitude', 'from_station_longitude']] = divvy['from_station_name'].apply(gps_lookup)
divvy[['to_station_latitude', 'to_station_longitude']] = divvy['to_station_name'].apply(gps_lookup)

In [ ]:
divvy

In [ ]:
divvy.to_csv('data/divvy_2017.csv')

In [ ]:
divvy = pd.read_csv('data/divvy_2017.csv')

### 1) Top 5 stations with the most starts (showing # of starts)

In [ ]:
station_starts = divvy.groupby(['from_station_name'])['from_station_name'].count().sort_values(ascending=False)
station_starts.head()

In [ ]:
ax = station_starts.head(5).plot(kind='bar', figsize=(15, 10), title='Top 5 Stations with Most Starts')
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005 + .15, p.get_height() * 1.005))

### 2) Trip duration by user type

In [ ]:
trip_duration = divvy.groupby(['usertype'])['tripduration'].mean().sort_values(ascending=False)
trip_duration

In [ ]:
ax = divvy.boxplot(column='tripduration', by='usertype', figsize=(15,10))

In [ ]:
ax = divvy.boxplot(column='tripduration', by='usertype', figsize=(15,10), showfliers=False)

### 3) Most popular trips based on start station and stop station

In [ ]:
divvy['trip_stations'] = divvy['from_station_name'] + ' TO ' + divvy['to_station_name']

In [ ]:
trip_stations = divvy.groupby(['trip_stations'])['trip_stations'].count().sort_values(ascending=False)
trip_stations.head()

In [ ]:
ax = trip_stations.head(10).plot(kind='bar', figsize=(15, 10), title='Top 10 Most Popular Trips')
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005 + .05, p.get_height() * 1.005))

### Create dataframe of paths for Tableau chart

In [ ]:
trip_dict = {'path': list(trip_stations.index), 'frequency': list(trip_stations.values)}
trip_dict['origin'] = [x.split(' TO ')[0] for x in trip_dict['path']]
trip_dict['destination'] = [x.split(' TO ')[1] for x in trip_dict['path']]

In [ ]:
from collections import defaultdict

trips = defaultdict(list)

for idx in range(len(trip_dict['path'])):
    trips['path'].append(trip_dict['path'][idx])
    trips['frequency'].append(trip_dict['frequency'][idx])
    trips['origin-destination'].append('origin')
    trips['station'].append(trip_dict['origin'][idx])
    
    trips['path'].append(trip_dict['path'][idx])
    trips['frequency'].append(trip_dict['frequency'][idx])
    trips['origin-destination'].append('destination')
    trips['station'].append(trip_dict['destination'][idx])

In [ ]:
stations_gps = (divvy[divvy['from_station_name'].duplicated()]
                [['from_station_name', 'from_station_latitude', 'from_station_longitude']].reset_index(drop=True))

In [ ]:
trips_df = pd.DataFrame(trips).merge(stations_gps.drop_duplicates(subset=['from_station_name']),
                                     how='left',
                                     left_on='station',
                                     right_on='from_station_name')

In [ ]:
trips_df.drop(['from_station_name'], axis=1, inplace=True)

In [ ]:
trips_df.to_csv('data/trips.csv')

### 4) Rider performance by Gender and Age based on avg trip distance (station to station), median speed (distance traveled / trip duration)

Multiply geodesic distance by 1.25. Routes follow roads but the calculated route is direct (geodesic). A route straight down a road would be the same as the direct route; a route diagnoal to roads would be multiplied by 1.414 (thanks, Pythagoras!); assuming routes are evenly split between diagonal and direct, with some wiggle room, I'm splitting the difference at 1.25.

I looked at using the Google Maps api to calculate the actual, along-the-road distance, but they've removed the free api key option. I also looked at Bing Maps, but it's rate limited and I have more than 98,000 routes in this dataset (and once I saw how big that number was, I realized that using api calls would take more than a few days!). So I opted for this *x1.25* method which is less accurate but far quicker and cheaper.

In [ ]:
from math import isnan

def find_distance(row):
    if (not isnan(row['from_station_latitude']) and
        not isnan(row['from_station_longitude']) and
        not isnan(row['to_station_latitude']) and
        not isnan(row['to_station_longitude'])):
        return (1.25 * (geopy.distance.distance((row['from_station_latitude'], row['from_station_longitude']),
                                                (row['to_station_latitude'], row['to_station_longitude'])).m))
    return np.nan

In [ ]:
from math import isnan

def find_distance(row):
    if (not isnan(row['from_station_latitude']) and
        not isnan(row['from_station_longitude']) and
        not isnan(row['to_station_latitude']) and
        not isnan(row['to_station_longitude'])):
        distance = (1.25 * (geopy.distance.distance((row['from_station_latitude'], row['from_station_longitude']),
                                                (row['to_station_latitude'], row['to_station_longitude'])).m))
        print(distance)
        return distance
    else:
        print(np.nan)
    return np.nan

In [ ]:
divvy['trip_distance'] = divvy.apply(find_distance, axis=1)
divvy['speed'] = divvy['trip_distance'] / divvy['tripduration']

In [ ]:
divvy.to_csv('data/divvy_2017.csv')

In [ ]:
divvy = pd.read_csv('data/divvy_2017.csv')

# Yelp data

In [ ]:
# with open('data/yelp_dataset/yelp_academic_dataset_business.json', 'rb') as f:
#     business = json.load(f)
    
with open('data/yelp_dataset/yelp_academic_dataset_checkin.json') as f:
    checkin = json.load(f)
    
# with open('data/yelp_dataset/yelp_academic_dataset_review.json') as f:
#     review = json.load(f)

In [ ]:
def open_yelp_data(filename):
    # read the entire file into a python array
    with open(filename, 'rb') as f:
        data = f.readlines()

    # remove the trailing "\n" from each line
    data = [line.rstrip() for line in data]

    data_json_str = "[" + ','.join(data) + "]"

    # now, load it into pandas
    data_df = pd.read_json(data_json_str)
    
    return data_df

In [ ]:
review = open_yelp_data('data/yelp_dataset/yelp_academic_dataset_review.json')

In [ ]:
with open('data/yelp_dataset/yelp_academic_dataset_checkin.json', 'rb') as f:
    data = f.readlines()

In [ ]:
data = [line.rstrip() for line in data]

In [ ]:
data

In [ ]:
data_json_str = "[" + ','.join(data) + "]"

# Using Kaggle data

In [ ]:
# divvy = pd.read_csv('data/chicago-divvy-bicycle-sharing-data/data_raw.csv', parse_dates=['starttime', 'stoptime'])

In [ ]:
# divvy = divvy[divvy['starttime'].dt.year == 2017].reset_index(drop=True)

In [ ]:
# divvy.to_csv('data/divvy_2017_kaggle.csv')

In [ ]:
divvy = pd.read_csv('data/divvy_2017_kaggle.csv')

In [ ]:
divvy

In [ ]:
divvy.head()

In [ ]:
divvy.columns

### 1) Top 5 stations with the most starts (showing # of starts)

In [ ]:
station_starts = divvy.groupby(['from_station_name'])['from_station_name'].count().sort_values(ascending=False)
station_starts.head()

In [ ]:
ax = station_starts.head(5).plot(kind='bar', figsize=(15, 10), title='Top 5 Stations with Most Starts')
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005 + .15, p.get_height() * 1.005))

In [ ]:
station_coord = 

In [ ]:
map = Basemap(width=10000000,height=6000000,projection='lcc',
            resolution=None,lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
plt.figure(figsize=(19,20))
map.bluemarble()

In [ ]:
for station in station_starts[5]:
        loc = geolocator.geocode(city)
        if not loc:
            print("Could not locate {}".format(city))
            continue
        x, y = map(loc.longitude, loc.latitude)
        map.plot(x,y,marker='o',color='Red',markersize=int(math.sqrt(count))*scale)
        plt.annotate(city, xy = (x,y), xytext=(-20,20)) 

### 2) Trip duration by user type

In [ ]:
trip_duration = divvy.groupby(['usertype'])['tripduration'].mean().sort_values(ascending=False)
trip_duration

In [ ]:
ax = divvy.boxplot(column='tripduration', by='usertype', figsize=(15,10))

In [ ]:
ax = divvy.boxplot(column='tripduration', by='usertype', figsize=(15,10), showfliers=False)

### 3) Most popular trips based on start station and stop station

In [ ]:
divvy['trip_stations'] = divvy['from_station_name'] + ' TO ' + divvy['to_station_name']

In [ ]:
trip_stations = divvy.groupby(['trip_stations'])['trip_stations'].count().sort_values(ascending=False)
trip_stations.head(10)

In [ ]:
ax = trip_stations.head(10).plot(kind='bar', figsize=(15, 10), title='Top 10 Most Popular Trips')
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005 + .05, p.get_height() * 1.005))

In [ ]:
trip_dict = {'path': list(trip_stations.index), 'frequency': list(trip_stations.values)}
trip_dict['origin'] = [x.split(' TO ')[0] for x in trip_dict['path']]
trip_dict['destination'] = [x.split(' TO ')[1] for x in trip_dict['path']]

In [ ]:
from collections import defaultdict

trips = defaultdict(list)

for idx in range(len(trip_dict['path'])):
    trips['path'].append(trip_dict['path'][idx])
    trips['origin-destination'].append('origin')
    trips['station'].append(trip_dict['origin'][idx])
    
    trips['path'].append(trip_dict['path'][idx])
    trips['origin-destination'].append('destination')
    trips['station'].append(trip_dict['destination'][idx])

In [ ]:
stations_gps = (divvy[divvy['from_station_name'].duplicated()]
                [['from_station_name', 'latitude_start', 'longitude_start']].reset_index(drop=True))

In [ ]:
trips_df = pd.DataFrame(trips).merge(stations_gps, how='left', left_on='station', right_on='from_station_name')

In [ ]:
trips_df.to_csv('data/trips.csv')

### 4) Rider performance by Gender and Age based on avg trip distance (station to station), median speed (distance traveled / trip duration)

Multiply geodesic distance by 1.25. Routes follow roads but the calculated route is direct (geodesic). A route straight down a road would be the same as the direct route; a route diagnoal to roads would be multiplied by 1.414 (thanks, Pythagoras!); assuming routes are evenly split between diagonal and direct, with some wiggle room, I'm splitting the difference at 1.25.

I looked at using the Google Maps api to calculate the actual, along-the-road distance, but they've removed the free api key option. I also looked at Bing Maps, but it's rate limited and I have more than 98,000 routes in this dataset (and once I saw how big that number was, I realized that using api calls would take more than a few days!). So I opted for this *x1.25* method which is less accurate but far quicker and cheaper.

In [ ]:
def find_distance(row):
    return (1.25 * (geopy.distance.distance((row['latitude_start'], row['longitude_start']),
                                            (row['latitude_end'], row['longitude_end'])).m))

In [ ]:
divvy['trip_distance'] = divvy.apply(find_distance, axis=1)

In [ ]:
divvy['speed'] = divvy['trip_distance'] / divvy['tripduration']

In [ ]:
divvy[['starttime', 'stoptime', 'tripduration', 'latitude_start', 'longitude_start', 'latitude_end', 'longitude_end', 'trip_distance', 'speed']]